# Problemas

Como vimos acima, há muitos passos na criação e definição de uma nova rede neural.
A grande parte desses ajustes dependem diretamente do problemas.

Abaixo, listamos alguns problemas. Todos os problemas e datasets usados vem do [Center for Machine Learning and Intelligent Systems](http://archive.ics.uci.edu/ml/datasets.php).


**Seu objetivo é determinar e implementar um modelo para cada problema.**

Isso inclui:

1. definir uma arquitetura.
Por enquanto usando somente camadas [Lineares](https://pytorch.org/docs/stable/nn.html#linear), porém podemos variar as ativações, como [Sigmoid](https://pytorch.org/docs/stable/nn.html#sigmoid), [Tanh](https://pytorch.org/docs/stable/nn.html#tanh), [ReLU](https://pytorch.org/docs/stable/nn.html#relu), [LeakyReLU](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html), [ELU](https://pytorch.org/docs/stable/generated/torch.nn.ELU.html), [SeLU](https://pytorch.org/docs/stable/generated/torch.nn.SELU.html), [PReLU](https://pytorch.org/docs/stable/generated/torch.nn.PReLU.html), [RReLU](https://pytorch.org/docs/stable/generated/torch.nn.RReLU.html)
2. definir uma função de custo. Algums opções que vimos previamente incluem[L1](https://pytorch.org/docs/stable/nn.html#l1loss), [L2/MSE](https://pytorch.org/docs/stable/nn.html#mseloss), [Huber/SmoothL1](https://pytorch.org/docs/stable/nn.html#smoothl1loss), [*Cross-Entropy*](https://pytorch.org/docs/stable/nn.html#crossentropyloss), [Hinge](https://pytorch.org/docs/stable/nn.html#hingeembeddingloss)), e
3. definir um algoritmo de otimização ([SGD](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD), [RMSProp](https://pytorch.org/docs/stable/optim.html#torch.optim.RMSprop), [Adam](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam))

A leitura do dado assim como a função de treinamento já estão implementados para você.

# Preâmbulo

In [2]:
# imports basicos
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torchvision

from torchvision import datasets, transforms
from torch import optim, nn

import os
import sys
import time
import numpy as np

In [3]:
import matplotlib.pyplot as plt
plt.ion()

In [4]:
# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids = list(range(n))
device

device(type='cuda')

## Funções básicas

Use a função `load_array ` declarada a seguir se voce ja tem os dados armazenados em um **array** (por exemplo um array do numpy, o `np.array`). Pode acontecer de que os nossos dados vêm simplesmente de um dataset que pode ser armazenado em um array, e portanto não é necessário fazer os outros passos mais complicados como carregar os dados do disco, etc; basta que possamos recuperar esses dados em *batches* aleatórios. O resultado dessa função é um `DataLoader` do Pytorch com os dados que fornecemos de entrada, e que permite que os acessamos da seguinte forma:

```python
data_loader = load_array(X, y, batch_size=32, is_train=True)
for x_batch, y_batch in data_loader:
    ### ... nossa iteração de treinamento aqui.
```

Essa função recebe como parâmetro os seguintes valores:

- `features`: um array que contém as features de todas as instâncias do dataset. Por exemplo, no caso do MNIST seria um array de tamanho `(60000, 28, 28, 1)` com todas as imagens do dataset de treino.
- `labels`: um array que contém os rótulos de cada instância de dados. No caso do MNIST, seria um array de tamanho `(60000,)` em que a posição `i` contém o rótulo do dígito da posição `i` do array `features`.
- `batch_size`: tamanho do batch desejado
- `is_train`: um booleano que indica se o dataset que estamos criando é o conjunto de treinamento ou não (conjunto de teste). A única mudança que isso causa no `Dataloader` resultante é que se for o conjunto de treinamento ele cria batches aleatórios.


In [5]:
def load_array(features, labels, batch_size, is_train=True):
    """Construct a Torch data loader"""

    ## transform the input arrays in a tensor in case they are not
    if type(features) != torch.tensor:
        features = torch.tensor(features)
    if type(labels) != torch.tensor:
        labels = torch.tensor(labels)

    ## create a Pytorch Dataset and DataLoader with the input data
    dataset = torch.utils.data.TensorDataset(features, labels)
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle=is_train)

Use a função `evaluate_accuracy` para calcular a acurácia e a *loss-function* para a rede em um conjunto de dados. Note que essa função pode ser usada tanto para avaliar a rede no conjunto de teste (no caso que usamos o `DataLoader` de teste) quanto o conjunto de treinamento (se usamos o `DataLoader` de treinamento). Os parâmetros são:
- `data_iter`: um `DataLoader` que contém os dados que queremos usar para avaliar a rede. Repare que esse parâmetro tipicamente é o ojeto que obtemos como saída da função `load_array` para montar o nosso `DataLoader`.
- `net`: a rede que queremos avaliar
- `loss`: a nossa *loss-function*. Pode ser um objeto de qualquer uma das funções de perda que mencionamos acima no começo do notebook.

O resultado dessa função é uma tupla em que o primeiro valor é a acurácia e o segundo a função de custo calculados.

In [6]:
# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    ## valores "acumuladores", que guardam a soma de, respectivamente, quantas instâncias
    ## prevemos corretamente, quantas instâncias percorremos no dataset, e o valor da loss; para
    ## todos os batches
    acc_sum, n, l = 0, 0, 0

    ## muda a rede para o "modo de teste". O que isso faz é mudar o comportamento de alguns módulos da rede,
    ## como os módulos de Dropout e BatchNorm, que funcionam de forma diferente quando estamos treinando ou
    ## quando estamos avaliando (ou usando em produção) a rede
    net.eval()

    with torch.no_grad():
      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y.long())

          ## aqui estamos calculando a quantidade de previsões que temos correta para o batch atual. o resultado
          ## do argmax é a posição de `y_hat` que possui o maior valor. Consequentemente isso resulta na classe que
          ## a rede deu o maior score.
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()

          ##
          n += y.size(0)

    return acc_sum / n, l.item() / len(data_iter)

A função `train_validate` é a função que implementa nossas iterações de treinamento padrão. Ela ja faz o trabalho de percorrer o dataset inteiro para cada época, e também de tempos em tempos avaliar a rede e mostar os resultados na tela. Para isso ela faz chamadas à função `evaluate_accuracy` declarada anteriormente (entre outras coisas). Essa função tem os segugintes parâmetros:
- `net`: a rede que queremos treinar
- `train_iter` e `test_iter`: nossos `DataLoaders` que criamos para acessar os dados. Esses DataLoaders podem ser criados com a função `load_array` declarada acima.
- `trainer`: é o nosso otimizador. Podemos usar aqui qualquer um dos otimizadores que escolhermos da lista citada no começo desse notebook.
- `loss`: a loss function que escolhemos para otimizar. Pode ser qualquer um das funções de custo citadas no começo do notebook.
- `num_epochs`: a quantidade de épocas pelas quais queremos que o treinamento ocorra.
- `type`: o tipo de tarefa que estamos lidando. Se for um problema de regressão, usamos `type='regression'`, e se for um problema de classificação, usamos `type='classification'`. Esse parâmetro é necessário para a função, por exemplo, saber quais métricas ele vai mostrar (acurácia, ou apenas o MSE, etc.)

In [7]:
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, trainer, loss, num_epochs, type='regression'):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            if type == 'regression':
              l = loss(y_hat, y.float())
            else:
              l = loss(y_hat, y.long())
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size(0)
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        if type == 'regression':
          print('epoch %d, train loss %.4f, test loss %.4f, time %.1f sec'
                % (epoch + 1, train_l_sum / len(train_iter), test_loss, time.time() - start))
        else:
          print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))


Use a função a seguir para inicializar os pesos da rede. Ela recebe como parâmetro um módulo da rede neural, e se for uma camada linear ele inicializa os pesos e os bias dessa camada. Embora possa parecer complicado de precisar chamar essa função para todas as camadas lineares da nossa rede, o módulos do Pytorch (que incluem tanto as redes criadas com o `nn.Sequential` ou com `nn.Module`) possuem a função `net.apply()` que recebe como parâmetro uma função e aplica ela a todos os submódulos da rede. Portanto, depois de ter criado a nossa rede, podemos chamar:

```python
net.apply(weights_init)
```
que automaticamente todas as camadas `nn.Linear` serão inicializadas. Caso queira saber mais sobre o `.apply()`, veja o seguinte [link](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.apply).

In [8]:
# Função para inicializar pesos da rede
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 0.01) # valores iniciais são uma normal
        m.bias.data.fill_(0)

# Problema 1

Neste problema, você receberá 14 *features* coletadas de pacientes e tentará predizer se eles tem algum sinal de doença cardíaca. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

In [10]:
## aqui fazemos o download do dataset usando o `!wget`. Se estamos rodando em um servidor linux (como é o caso do Colab),
## podemos usar comandos do linux precedidos pelo "!". Por exemplo podemos fazer !ls para listar os arquivos da instância do colab.
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data

## aqui fazemos um tratamento inicial dos dados. "np.genfromtxt" lê os dados de um arquivo .txt e transforma em
## um array. Pode ser interessante abrir o arquivo para verificar como os dados chegaram. Se estiver no colab, voce
## pode verificar o arquivo "processed.cleveland.data" clicando na pastinha do canto esquerdo da página. a função
## "np.nan_to_num" trata valores NaN e infinitos no dataset.
data = np.genfromtxt('processed.cleveland.data', delimiter=',', dtype=np.float32)
data = np.nan_to_num(data)

--2025-05-19 23:35:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘processed.cleveland.data’

processed.cleveland     [ <=>                ]  18,03K  95,5KB/s    in 0,2s    

2025-05-19 23:35:43 (95,5 KB/s) - ‘processed.cleveland.data’ saved [18461]



In [11]:
## aqui separamos os dados entre features (X) e rótulo (y), e depois separamos em um conjunto de treinamento e teste
print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0])
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.20, random_state=42)

(303, 14) [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   6.    0. ]
(303, 13) [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   6. ]
(303,) 0.0


In [12]:
## aqui criamos nossos DataLoaders para conseguirmos iterar nos dados
batch_size = 64
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

In [19]:
class HeartDiseaseNet(nn.Module):
    def __init__(self):
        super(HeartDiseaseNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(13, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 5),
        )

    def forward(self, x):
        return self.model(x)

In [20]:
model = HeartDiseaseNet().to(device)
model.apply(weights_init)

HeartDiseaseNet(
  (model): Sequential(
    (0): Linear(in_features=13, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=5, bias=True)
  )
)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
train_validate(model, train_iter, test_iter, optimizer, criterion, num_epochs=100, type='classification')

training on cuda
epoch 1, train loss 1.5948, train acc 0.471, test loss 1.5756, test acc 0.475, time 0.3 sec
epoch 2, train loss 1.5380, train acc 0.558, test loss 1.5070, test acc 0.475, time 0.0 sec
epoch 3, train loss 1.4359, train acc 0.558, test loss 1.4049, test acc 0.475, time 0.0 sec
epoch 4, train loss 1.3110, train acc 0.558, test loss 1.4083, test acc 0.475, time 0.0 sec
epoch 5, train loss 1.2652, train acc 0.558, test loss 1.4521, test acc 0.475, time 0.0 sec
epoch 6, train loss 1.2800, train acc 0.558, test loss 1.4055, test acc 0.475, time 0.0 sec
epoch 7, train loss 1.2623, train acc 0.558, test loss 1.3738, test acc 0.475, time 0.0 sec
epoch 8, train loss 1.2701, train acc 0.558, test loss 1.3714, test acc 0.475, time 0.0 sec
epoch 9, train loss 1.2586, train acc 0.558, test loss 1.3708, test acc 0.475, time 0.0 sec
epoch 10, train loss 1.2391, train acc 0.558, test loss 1.3755, test acc 0.475, time 0.0 sec
epoch 11, train loss 1.2426, train acc 0.558, test loss 1.3864

# Problema 2

Neste problema, você receberá 90 *features* extraídas de diversas músicas (datadas de 1922 até 2011) e deve predizer o ano de cada música. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD

In [14]:
# download do dataset
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
!unzip YearPredictionMSD.txt.zip
data = np.genfromtxt('YearPredictionMSD.txt', delimiter=',', dtype=np.float32)

print(data[0, :])
X, y = data[:, 1:], data[:, 0]
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)

batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

--2025-05-19 23:36:47--  http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘YearPredictionMSD.txt.zip’

YearPredictionMSD.t     [   <=>              ] 201,24M  3,42MB/s    in 49s     

2025-05-19 23:37:37 (4,13 MB/s) - ‘YearPredictionMSD.txt.zip’ saved [211011981]

Archive:  YearPredictionMSD.txt.zip
  inflating: YearPredictionMSD.txt   
[ 2.0010000e+03  4.9943569e+01  2.1471140e+01  7.3077499e+01
  8.7486095e+00 -1.7406281e+01 -1.3099050e+01 -2.5012020e+01
 -1.2232570e+01  7.8308902e+00 -2.4678299e+00  3.3213601e+00
 -2.3152101e+00  1.0205560e+01  6.1110913e+02  9.5108960e+02
  6.9811426e+02  4.0898486e+02  3.8370911e+02  3.2651511e+02
  2.3811327e+02  2.5142413e+02  1.8717351e+02  1.0042652e+02
  1.7919498e+02 -8

In [15]:
# Define the neural network architecture
class MusicYearNet(nn.Module):
    def __init__(self):
        super(MusicYearNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(90, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)

model_2 = MusicYearNet().to(device)
model_2.apply(weights_init)

criterion_2 = nn.MSELoss()
optimizer_2 = optim.Adam(model_2.parameters(), lr=0.001)

train_validate(model_2, train_iter, test_iter, optimizer_2, criterion_2, num_epochs=50, type='regression')

training on cuda


/home/thiago/UFMG/10_periodo/deep_learning/deep_learning/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/thiago/UFMG/10_periodo/deep_learning/deep_learning/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([81])) that is different to the input size (torch.Size([81, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/thiago/UFMG/10_periodo/deep_learning/deep_learning/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([64])) that is different to the input

epoch 1, train loss 92139.9850, test loss 28725.0535, time 9.9 sec
epoch 2, train loss 20459.4988, test loss 17078.7537, time 9.3 sec
epoch 3, train loss 17727.5458, test loss 14588.3951, time 9.1 sec
epoch 4, train loss 16045.7524, test loss 20276.7737, time 10.4 sec
epoch 5, train loss 15132.9285, test loss 12680.7102, time 11.5 sec
epoch 6, train loss 14015.1948, test loss 12705.3004, time 12.2 sec
epoch 7, train loss 13555.5597, test loss 16629.1052, time 13.0 sec
epoch 8, train loss 12971.2334, test loss 13823.1675, time 10.6 sec
epoch 9, train loss 12409.2308, test loss 11535.8154, time 11.3 sec
epoch 10, train loss 12042.3070, test loss 12340.0364, time 11.5 sec
epoch 11, train loss 11690.8348, test loss 10617.5873, time 9.9 sec
epoch 12, train loss 11289.9580, test loss 10643.3239, time 10.1 sec
epoch 13, train loss 11075.4329, test loss 12171.4039, time 10.1 sec
epoch 14, train loss 10754.5068, test loss 9933.9553, time 11.4 sec
epoch 15, train loss 10451.7050, test loss 10588

In [16]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = model_2(torch.Tensor(test_features[0:5, :]).to(device))
print(y, test_labels[0:5])

tensor([[1998.2603],
        [1998.2603],
        [1998.2603],
        [1998.2603],
        [1998.2603]], device='cuda:0', grad_fn=<AddmmBackward0>) [2008. 2001. 2006. 2008. 1998.]


# Problema 3

Neste problema, você receberá várias *features* (como altura média, inclinação, etc) descrevendo uma região e o modelo deve predizer qual o tipo da região (floresta, montanha, etc). Mais informações sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/covertype

In [17]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
!gzip covtype.data.gz
data = np.genfromtxt('covtype.data', delimiter=',', dtype=np.float32)

print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0])
train_features_3, test_features_3, train_labels_3, test_labels_3 = train_test_split(X, y, test_size=0.33, random_state=42)
train_labels_3 = train_labels_3 - 1
test_labels_3 = test_labels_3 - 1

batch_size = 100
train_iter_3 = load_array(train_features_3, train_labels_3, batch_size)
test_iter_3 = load_array(test_features_3, test_labels_3, batch_size, False)

--2025-05-19 23:49:50--  http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘covtype.data.gz’

covtype.data.gz         [               <=>  ]  10,72M  2,90MB/s    in 4,0s    

2025-05-19 23:49:54 (2,65 MB/s) - ‘covtype.data.gz’ saved [11240707]

gzip: covtype.data.gz already has .gz suffix -- unchanged
(581012, 55) [2.596e+03 5.100e+01 3.000e+00 2.580e+02 0.000e+00 5.100e+02 2.210e+02
 2.320e+02 1.480e+02 6.279e+03 1.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 

In [18]:
# Define the neural network architecture
class CoverTypeNet(nn.Module):
    def __init__(self):
        super(CoverTypeNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(54, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 7),
        )

    def forward(self, x):
        return self.model(x)

model_3 = CoverTypeNet().to(device)
model_3.apply(weights_init)

criterion_3 = nn.CrossEntropyLoss()
optimizer_3 = optim.Adam(model_3.parameters(), lr=0.001)

train_validate(model_3, train_iter_3, test_iter_3, optimizer_3, criterion_3, num_epochs=50, type='classification')

training on cuda
epoch 1, train loss 0.7759, train acc 0.664, test loss 0.7032, test acc 0.694, time 13.0 sec
epoch 2, train loss 0.6548, train acc 0.716, test loss 0.6413, test acc 0.731, time 12.6 sec
epoch 3, train loss 0.6150, train acc 0.733, test loss 0.5831, test acc 0.750, time 13.8 sec
epoch 4, train loss 0.5896, train acc 0.744, test loss 0.5842, test acc 0.742, time 12.6 sec
epoch 5, train loss 0.5647, train acc 0.756, test loss 0.5507, test acc 0.767, time 13.6 sec
epoch 6, train loss 0.5475, train acc 0.764, test loss 0.5686, test acc 0.756, time 13.5 sec
epoch 7, train loss 0.5318, train acc 0.771, test loss 0.5139, test acc 0.778, time 13.4 sec
epoch 8, train loss 0.5217, train acc 0.776, test loss 0.5097, test acc 0.782, time 11.7 sec
epoch 9, train loss 0.5115, train acc 0.781, test loss 0.5183, test acc 0.778, time 11.6 sec
epoch 10, train loss 0.5045, train acc 0.785, test loss 0.5197, test acc 0.772, time 11.7 sec
epoch 11, train loss 0.4972, train acc 0.788, test l

In [19]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = model_3(torch.Tensor(test_features_3[0:5, :]).to(device))
print(y, test_labels_3[0:5])

tensor([[ 7.5493e+00,  1.1596e+00, -3.8146e+01, -3.5614e+02, -1.7936e+01,
         -4.6732e+01,  4.0308e+00],
        [-4.6020e-01,  2.4049e+00, -1.8511e+00, -3.1516e+01, -6.9974e-01,
         -4.0119e+00, -1.2465e+01],
        [-3.2605e+00,  2.1820e+00,  1.0269e+00, -9.0699e+00, -8.5586e-01,
         -2.0352e+00, -1.4700e+01],
        [ 1.9515e+00,  2.4313e+00, -2.1547e+01, -2.2555e+02, -2.6122e+00,
         -1.8438e+01, -1.9116e+01],
        [ 2.7471e-01,  3.4888e+00, -8.1679e+00, -9.8457e+01,  2.2671e-01,
         -7.8202e+00, -3.5766e+01]], device='cuda:0', grad_fn=<AddmmBackward0>) [0. 1. 1. 1. 1.]


# Problema 4

Neste problema, você receberá 11 *features* extraídas de tipos de vinhos, e terá que predizer um *score* para cada vinho. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

In [22]:
# download do dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
data_red = np.genfromtxt('winequality-red.csv', delimiter=';', dtype=np.float32, skip_header=1)
data_white = np.genfromtxt('winequality-white.csv', delimiter=';', dtype=np.float32, skip_header=1)
data = np.concatenate((data_red, data_white), axis=0)
data = np.nan_to_num(data)

print(data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
train_features_4, test_features_4, train_labels_4, test_labels_4 = train_test_split(X, y, test_size=0.33, random_state=42)

batch_size = 100
train_iter = load_array(train_features_4, train_labels_4, batch_size)
test_iter = load_array(test_features_4, test_labels_4, batch_size, False)

--2025-05-20 00:05:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-red.csv.1’

winequality-red.csv     [  <=>               ]  82,23K   223KB/s    in 0,4s    

2025-05-20 00:05:25 (223 KB/s) - ‘winequality-red.csv.1’ saved [84199]

--2025-05-20 00:05:25--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-white.csv.1’

winequality-white.c     [   <=>              ] 258,23K   341KB/s    in 0,8s    

2025

In [23]:
class WineQualityNet(nn.Module):
    def __init__(self):
        super(WineQualityNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(11, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

model_4 = WineQualityNet().to(device)
model_4.apply(weights_init)

criterion = nn.MSELoss()
optimizer = optim.Adam(model_4.parameters(), lr=0.001)

train_validate(model_4, train_iter, test_iter, optimizer, criterion, num_epochs=50, type='regression')

training on cuda
epoch 1, train loss 19.6759, test loss 5.9013, time 0.2 sec


/home/thiago/UFMG/10_periodo/deep_learning/deep_learning/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/thiago/UFMG/10_periodo/deep_learning/deep_learning/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([52])) that is different to the input size (torch.Size([52, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/thiago/UFMG/10_periodo/deep_learning/deep_learning/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([45])) that is different to the input

epoch 2, train loss 4.8493, test loss 4.1678, time 0.2 sec
epoch 3, train loss 3.2795, test loss 2.4743, time 0.1 sec
epoch 4, train loss 1.6250, test loss 0.9955, time 0.1 sec
epoch 5, train loss 0.9159, test loss 0.8656, time 0.1 sec
epoch 6, train loss 0.8865, test loss 0.8533, time 0.1 sec
epoch 7, train loss 0.8844, test loss 0.8490, time 0.1 sec
epoch 8, train loss 0.8794, test loss 0.8507, time 0.1 sec
epoch 9, train loss 0.8798, test loss 0.8428, time 0.1 sec
epoch 10, train loss 0.8738, test loss 0.8399, time 0.2 sec
epoch 11, train loss 0.8739, test loss 0.8400, time 0.1 sec
epoch 12, train loss 0.8728, test loss 0.8375, time 0.1 sec
epoch 13, train loss 0.8677, test loss 0.8335, time 0.1 sec
epoch 14, train loss 0.8697, test loss 0.8423, time 0.1 sec
epoch 15, train loss 0.8624, test loss 0.8392, time 0.1 sec
epoch 16, train loss 0.8648, test loss 0.8260, time 0.2 sec
epoch 17, train loss 0.8739, test loss 0.8267, time 0.1 sec
epoch 18, train loss 0.8895, test loss 0.8723, t

In [24]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = model_4(torch.Tensor(test_features_4[0:5, :]).to(device))
print(y, test_labels_4[0:5])

tensor([[6.0086],
        [5.8021],
        [5.9521],
        [5.5954],
        [5.7929]], device='cuda:0', grad_fn=<AddmmBackward0>) [8. 5. 7. 6. 6.]
